## Fixed Features
This notebook demonstrates computing several technical indicators such as Stochastic Oscillator, Williams %R, MACD, Bollinger Bands (%b and bandwidth), ADX, CCI and On-Balance Volume along with other momentum and volatility features.

In [ ]:

import pandas as pd
from src.features.technical_indicators import compute_technical_indicators
from src.features.additional_features import compute_additional_features

df = pd.read_parquet("../../data/06data.parquet")

df = compute_technical_indicators(df)
df = compute_additional_features(df)

cols = ['stoch_k','williams_r','macd','macd_signal','proc',
        'bollinger_b','bollinger_bw','adx','cci','obv']
print(df[cols].head())
